<a href="https://colab.research.google.com/github/iued-uni-heidelberg/projTermiKnowledge/blob/main/TermiKnowledgeDemo03ExtractingTerms.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# This notebook show some methods of terminology extraction

1. Installing Part-of-Speech tagger for languages we need:

... English...

2. Uploading files

In [ ]:
%%bash
mkdir treetagger
cd treetagger
# Download the tagger package for your system (PC-Linux, Mac OS-X, ARM64, ARMHF, ARM-Android, PPC64le-Linux).
wget https://cis.lmu.de/~schmid/tools/TreeTagger/data/tree-tagger-linux-3.2.4.tar.gz
tar -xzvf tree-tagger-linux-3.2.4.tar.gz
# Download the tagging scripts into the same directory.
wget https://cis.lmu.de/~schmid/tools/TreeTagger/data/tagger-scripts.tar.gz
gunzip tagger-scripts.tar.gz
# Download the installation script install-tagger.sh.
wget https://cis.lmu.de/~schmid/tools/TreeTagger/data/install-tagger.sh
# Download the parameter files for the languages you want to process.
# list of all files (parameter files) https://cis.lmu.de/~schmid/tools/TreeTagger/#parfiles
wget https://cis.lmu.de/~schmid/tools/TreeTagger/data/english.par.gz
sh install-tagger.sh
cd ..
sudo pip install treetaggerwrapper

In [ ]:
%%bash
wget https://heibox.uni-heidelberg.de/f/95a3875771c040db959a/?dl=1
mv index.html?dl=1 humanrights02.txt

wget https://heibox.uni-heidelberg.de/f/a79b829e15c24dbd9e77/?dl=1
mv index.html?dl=1 covid3m-en.txt

wget https://heibox.uni-heidelberg.de/f/e3c1edbcec9649f5b4c4/?dl=1
mv index.html?dl=1 TED2020-enka-en.txt


In [ ]:
!wget https://heibox.uni-heidelberg.de/f/cdf240db84ca4718b718/?dl=1
!mv index.html?dl=1 go1984.txt

In [ ]:
!head --lines=20 humanrights02.txt
!wc humanrights02.txt

In [ ]:
!head --lines=20 TED2020-enka-en.txt
!wc TED2020-enka-en.txt

In [ ]:
!head --lines=20 covid3m-en.txt
!wc covid3m-en.txt

In [ ]:
!./treetagger/cmd/tree-tagger-english humanrights02.txt >humanrights02_vert.txt

In [ ]:
!./treetagger/cmd/tree-tagger-english TED2020-enka-en.txt >TED2020-enka-en_vert.txt

In [ ]:
!./treetagger/cmd/tree-tagger-english go1984.txt >go1984_vert.txt

In [ ]:
!./treetagger/cmd/tree-tagger-english covid3m-en.txt >covid3m-en_vert.txt

In [ ]:
!head --lines=20 humanrights02_vert.txt

In [ ]:
!head --lines=20 TED2020-enka-en_vert.txt

In [ ]:
!head --lines=20 covid3m-en_vert.txt

## Python script for term extraction

Now we use Part of Speech patterns + frequency information to rank potential candidates

In [15]:
import os, re, sys, time
def printDict(DictionaryFrq, FileOut):
    for Word, Frq in sorted( DictionaryFrq.items() , key=lambda x: x[1], reverse=True):
        if re.search(' ', Word):
            FileOut.write(Word + '\t' + str(Frq) + '\n')


def openFiles(SFIn, SFOut):
    FInStream = open(SFIn, 'r')
    FOutStream = open(SFOutn, 'w')
    return FInStream, FOutStream


class clProcCorpus(object):
    ''' we will read a text file and return a dictionary
    this will be done on the line by line basis
    The dictionary can be sorted later...
    '''
    # this is a class for processing a corpus

    def __init__(self, FileIN):
        self.DictFrq = {}
        self.processCorpus(FileIN)

    def processCorpus(self, FileIN):
        LTerm = []
        for Line in FileIN:
            Line = Line.strip()
            LLine = re.split('\t', Line)
            try:
                Word = LLine[0]
                PoS = LLine[1]
                Lemma = LLine[2]
            except:
                Word = ""
                PoS = ""
                Lemma = ""
            
      #Select the Tags for your langauge
            #if re.match('N.*', PoS) or re.match('A.*', PoS): #Arm
            #if re.match('N.*', PoS) or re.match('ADJ.*', PoS): #DE
            if re.match('N.*', PoS) or re.match('J.*', PoS): #EN

      #Terms as Words or Lemmas
                LTerm.append(Word)
                # LTerm.append(Lemma)
            else:
                STerm = ' '.join(LTerm)
                LTerm = []

                try:
                    self.DictFrq[STerm] += 1
                except:
                    self.DictFrq[STerm] = 1        

        return


In [ ]:
FIn, FOut = openFiles('humanrights02_vert.txt', 'humanrights02_terms.txt')